This file is a continuation of my data cleaning / joining after I found easier data to work with.

In [30]:
import pandas as pd
import geopandas as gpd
from shapely import Point

In [31]:
census_df = pd.read_csv("../data/nyc_decennialcensusdata_2010_2020_change-core-geographies_v2.csv")

census_df.head()

,Year,GeoType,Borough,GeoID,BCT2020,Name,CDType,NTAType,Pop1,Pop1P,...,ROcHH2,ROcHH2P,ROcHH3,ROcHH3P,ROcHH4,ROcHH4P,ROcHH5pl,ROcHH5plP,LandAcres,LandAcresP
0,2020,NYC2020,New York City,0,NaN,NYC,NaN,NaN,"8,804,190",100.0,...,"670,144",28.3,"374,447",15.8,"263,906",11.2,"254,065",10.7,"192,289",NaN
1,2020,Boro2020,Manhattan,1,NaN,Manhattan,NaN,NaN,"1,694,251",100.0,...,"196,690",31.4,"75,343",12.0,"43,007",6.9,"27,660",4.4,"14,500",NaN
2,2020,Boro2020,Bronx,2,NaN,Bronx,NaN,NaN,"1,472,654",100.0,...,"102,118",24.0,"77,071",18.1,"58,043",13.7,"61,014",14.4,"26,990",NaN
3,2020,Boro2020,Brooklyn,3,NaN,Brooklyn,NaN,NaN,"2,736,074",100.0,...,"215,925",29.1,"122,906",16.6,"84,476",11.4,"89,012",12.0,"44,401",NaN
4,2020,Boro2020,Queens,4,NaN,Queens,NaN,NaN,"2,405,464",100.0,...,"139,627",27.6,"89,400",17.6,"70,538",13.9,"68,090",13.4,"69,583",NaN


So happy to have this cleaned, easy data to work with. I need all rows where GeoType == NTA2020, then I can join it with the geojson file on that column and I'll have my population data like I wanted.

There is plenty of other data about people in these neighborhoods, which may have been of use, but that could be for another time. 

In [32]:
census_clean = census_df.loc[census_df["GeoType"] == "NTA2020", ["GeoID", "Pop1"]]
census_clean.rename(columns={"GeoID": "NTA2020", "Pop1": "Population"}, inplace=True)
census_clean.head()

,NTA2020,Population
128,BK0101,"38,980"
129,BK0102,"64,444"
130,BK0103,"47,703"
131,BK0104,"52,998"
132,BK0201,"25,092"


Great. This should be fine to merge with the GeoJSON.

In [33]:
geo_nta = gpd.read_file("../data/NYC_Neighborhood_Tabulation_Areas_2020.geojson")
geo_nta.head()

,OBJECTID,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,geometry
0,1,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.93214 40.72817, -73.93238 40.728..."
1,2,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.95814 40.72441, -73.95772 40.724..."
2,3,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.95024 40.70548, -73.94984 40.705..."
3,4,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.92406 40.71412, -73.92404 40.714..."
4,5,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,"POLYGON ((-73.99237 40.6897, -73.99436 40.6902..."


In [34]:
merged = geo_nta.merge(census_clean, on="NTA2020", how="left")
merged.head()

,OBJECTID,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,geometry,Population
0,1,3,Brooklyn,047,BK0101,Greenpoint,Grnpt,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.93214 40.72817, -73.93238 40.728...","38,980"
1,2,3,Brooklyn,047,BK0102,Williamsburg,Wllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.95814 40.72441, -73.95772 40.724...","64,444"
2,3,3,Brooklyn,047,BK0103,South Williamsburg,SWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.95024 40.70548, -73.94984 40.705...","47,703"
3,4,3,Brooklyn,047,BK0104,East Williamsburg,EWllmsbrg,0,BK01,BK01 Williamsburg-Greenpoint (CD 1 Equivalent),"POLYGON ((-73.92406 40.71412, -73.92404 40.714...","52,998"
4,5,3,Brooklyn,047,BK0201,Brooklyn Heights,BkHts,0,BK02,BK02 Downtown Brooklyn-Fort Greene (CD 2 Appro...,"POLYGON ((-73.99237 40.6897, -73.99436 40.6902...","25,092"


The relief this finally worked! Now to calculate Airbnb per person.

In [35]:
airbnb = pd.read_csv("../data/airbnb_one_year.csv")
airbnb.head()

,id,name,host_id,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_nights,availability_365,license,dataset_date
0,40824219,Room close to Manhattan for FEMALE guests,317540555,Sunnyside,Queens,40.746980,-73.91763,Private room,$66.00,30,77,NaN,2025-10-01
1,808629897642520802,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$330.00,30,285,NaN,2025-10-01
2,808629343999219473,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,285,NaN,2025-10-01
3,808629391209329400,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752660,-73.97248,Entire home/apt,NaN,30,286,NaN,2025-10-01
4,808629522066886810,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,285,NaN,2025-10-01


In [36]:
airbnb.rename(columns={"neighbourhood_cleansed": "neighborhood_cleansed", "neighbourhood_group_cleansed": "neighborhood_group_cleansed"},
                inplace=True)

In [37]:
"""
ChatGPT Citation:
I asked "I want to count how many Airbnbs are in a certain NTA. I have the lat and long for each Airbnb, as well as the 
shape file for each neighborhood. How would I do it?
This code was written with the help of its answer.
"""

# create shapely point for each Airbnb 
airbnb["geometry"] = airbnb.apply(lambda row: Point(row["longitude"], row["latitude"]), axis=1)


airbnb_gdf = gpd.GeoDataFrame(airbnb, geometry="geometry", crs="EPSG:4326")

# make sure merged gdf uses same crs
merged = merged.to_crs("EPSG:4326")

# spatial join, assigning each Airbnb to the NTA it falls inside of
joined = gpd.sjoin(airbnb_gdf, merged, how="left", predicate="within")

# count how many are in each NTA
airbnb_counts = joined.groupby("NTA2020").size().reset_index(name="airbnb_count")

airbnb_counts.head()

,NTA2020,airbnb_count
0,BK0101,654
1,BK0102,1431
2,BK0103,48
3,BK0104,1081
4,BK0201,126


In [38]:
joined.head()

,id,name,host_id,neighborhood_cleansed,neighborhood_group_cleansed,latitude,longitude,room_type,price,minimum_nights,...,BoroCode,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Population
0,40824219,Room close to Manhattan for FEMALE guests,317540555,Sunnyside,Queens,40.746980,-73.91763,Private room,$66.00,30,...,4.0,Queens,081,QN0202,Sunnyside,Snnysd,0,QN02,QN02 Long Island City-Sunnyside-Woodside (CD 2...,"52,278"
1,808629897642520802,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$330.00,30,...,1.0,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765"
2,808629343999219473,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,...,1.0,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765"
3,808629391209329400,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752660,-73.97248,Entire home/apt,NaN,30,...,1.0,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765"
4,808629522066886810,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,...,1.0,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765"


Almost there. Now to join the Airbnb counts with the geo dataframe, and calculate the number of Airbnbs based on population.

In [39]:
NTA_geo_pop = joined.merge(airbnb_counts, on="NTA2020", how="left")

NTA_geo_pop.head()

,id,name,host_id,neighborhood_cleansed,neighborhood_group_cleansed,latitude,longitude,room_type,price,minimum_nights,...,BoroName,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Population,airbnb_count
0,40824219,Room close to Manhattan for FEMALE guests,317540555,Sunnyside,Queens,40.746980,-73.91763,Private room,$66.00,30,...,Queens,081,QN0202,Sunnyside,Snnysd,0,QN02,QN02 Long Island City-Sunnyside-Woodside (CD 2...,"52,278",337.0
1,808629897642520802,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$330.00,30,...,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765",838.0
2,808629343999219473,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,...,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765",838.0
3,808629391209329400,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752660,-73.97248,Entire home/apt,NaN,30,...,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765",838.0
4,808629522066886810,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,...,Manhattan,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,"45,765",838.0


I will use Airbnbs per 10,000, since NTA population sizes are not that large, around 50k people each. 

In [40]:
# drop columns with NaN for population
NTA_geo_pop = NTA_geo_pop.dropna(subset=["Population"])

# remove commas from Population
NTA_geo_pop["Population"] = NTA_geo_pop["Population"].str.replace(",", "")

# fix data type
NTA_geo_pop["Population"] = NTA_geo_pop["Population"].astype(int)

In [41]:
NTA_geo_pop["airbnb_per_10k"] = round(NTA_geo_pop["airbnb_count"] / NTA_geo_pop["Population"] * 10000, 2)

In [42]:
NTA_geo_pop.head()

,id,name,host_id,neighborhood_cleansed,neighborhood_group_cleansed,latitude,longitude,room_type,price,minimum_nights,...,CountyFIPS,NTA2020,NTAName,NTAAbbrev,NTAType,CDTA2020,CDTAName,Population,airbnb_count,airbnb_per_10k
0,40824219,Room close to Manhattan for FEMALE guests,317540555,Sunnyside,Queens,40.746980,-73.91763,Private room,$66.00,30,...,081,QN0202,Sunnyside,Snnysd,0,QN02,QN02 Long Island City-Sunnyside-Woodside (CD 2...,52278,337.0,64.46
1,808629897642520802,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$330.00,30,...,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,45765,838.0,183.11
2,808629343999219473,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,...,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,45765,838.0,183.11
3,808629391209329400,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752660,-73.97248,Entire home/apt,NaN,30,...,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,45765,838.0,183.11
4,808629522066886810,Wyndham Midtown 45 Resort | King Bed Studio Suite,442029804,Midtown,Manhattan,40.752656,-73.97248,Entire home/apt,$312.00,30,...,061,MN0604,East Midtown-Turtle Bay,EstMdtwn,0,MN06,MN06 East Midtown-Murray Hill (CD 6 Approximat...,45765,838.0,183.11


I was having issues with extremely large values for Airbnb per 10k. Let me drop anything over 1000 and see if that helps.

The issue may be population as well, for things like parks or airports. Anything under 1000 will get dropped here as well. 

In [43]:
# drop high values for airbnb per 10k
NTA_geo_pop = NTA_geo_pop[NTA_geo_pop["airbnb_per_10k"] <= 1000]

# drop low values for population
NTA_geo_pop = NTA_geo_pop[NTA_geo_pop["Population"] >= 1000]

In [44]:
sorted_airbnb_10k = NTA_geo_pop["airbnb_per_10k"].sort_values(ascending=False).head()

sorted_airbnb_10k

45220    607.23
38061    607.23
38059    607.23
17004    607.23
42691    607.23
Name: airbnb_per_10k, dtype: float64

Also having an issue where Plotly was not recognizing values for NTA2020. It was because there were multiple values for each NTA2020, since each Airbnb had a NTA2020 value. So, I can just make another dataframe with only the NTA2020 and calculated Airbnb per 10k. I also had to include what I want my hover text to be, so I added NTA name and neighborhood group cleansed (borough name).

In [47]:
NTA_geo_pop["NTA2020"].value_counts().head(10)

NTA2020
MN0402    1699
MN0502    1553
BK0302    1528
BK0102    1431
BK0301    1420
MN0303    1244
MN0401    1218
BK0104    1081
BK0802    1078
MN0603     974
Name: count, dtype: int64

In [62]:
NTA_airbnb_10k = NTA_geo_pop.groupby("NTA2020")[["airbnb_per_10k", "NTAName", "neighborhood_group_cleansed"]].first().reset_index()

NTA_airbnb_10k.head()

,NTA2020,airbnb_per_10k,NTAName,neighborhood_group_cleansed
0,BK0101,167.78,Greenpoint,Brooklyn
1,BK0102,222.05,Williamsburg,Brooklyn
2,BK0103,10.06,South Williamsburg,Brooklyn
3,BK0104,203.97,East Williamsburg,Brooklyn
4,BK0201,50.22,Brooklyn Heights,Brooklyn


In [ ]:
# save as .csv for getting the points and stats calculated earlier
# or else it is a mess using a GeoJSON for it
NTA_geo_pop.drop(columns="geometry").to_csv("NTA_stats.csv", index=False)

# and now the .csv for Airbnb per 10k values as well
NTA_airbnb_10k.to_csv("NTA_airbnb_10k.csv")